In [4]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps

def load_images_from_folder(folder):
    images = []
    # Only process files with these image file extensions
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']
    for filename in os.listdir(folder):
        # Check the file extension before opening
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            img_path = os.path.join(folder, filename)
            with Image.open(img_path) as img:
                img = img.convert("RGBA")
                images.append((img, filename, folder.split(os.sep)[-1]))
    return images

def check_overlap(new_box, boxes):
    for box in boxes:
        if (new_box[0] < box[2] and new_box[2] > box[0] and
            new_box[1] < box[3] and new_box[3] > box[1]):
            return True
    return False

def apply_transformations(img):
    img = img.rotate(random.randint(-45, 45), expand=True)
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.7, 1.3))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.8, 1.2))
    if random.choice([True, False]):
        img = ImageOps.flip(img)
    if random.choice([True, False]):
        img = ImageOps.mirror(img)
    return img

def combine_images(image_list, background_size):
    background = Image.new('RGBA', background_size, (255, 255, 255, 255))
    annotations = []
    occupied_areas = []

    for img, filename, label in image_list:
        img = apply_transformations(img)
        scale_factor = min(background_size[0] / img.width, background_size[1] / img.height, 1)
        img = img.resize((int(img.width * scale_factor), int(img.height * scale_factor)), Image.LANCZOS)
        
        placed = False
        attempts = 0
        while not placed and attempts < 50:
            x, y = random.randint(0, background_size[0] - img.width), random.randint(0, background_size[1] - img.height)
            new_box = [x, y, x + img.width, y + img.height]
            if not check_overlap(new_box, occupied_areas):
                background.paste(img, (x, y), img)
                occupied_areas.append(new_box)
                annotations.append({'class': label, 'bbox': new_box, 'filename': filename})
                placed = True
            attempts += 1

    return background.convert('RGB'), annotations

def create_dataset(base_folder, num_images=200, min_items_per_image=5, max_items_per_image=10, fixed_size=(2048, 2048)):
    categories = [d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))]
    output_folder = os.path.join(base_folder, 'combined_dataset')
    os.makedirs(output_folder, exist_ok=True)

    for i in range(num_images):
        num_items = random.randint(min_items_per_image, max_items_per_image)
        selected_images = []
        selected_categories = random.sample(categories, min(4, len(categories)))

        while len(selected_images) < num_items:
            category_folder = random.choice(selected_categories)
            images = load_images_from_folder(os.path.join(base_folder, category_folder))
            if images:
                selected_image = random.choice(images)
                selected_images.append(selected_image)
                if len(selected_images) >= 4:
                    selected_categories = categories  # Allow any category after initial requirement

        combined_image, annotations = combine_images(selected_images, fixed_size)
        combined_image.save(os.path.join(output_folder, f'combined_{i}.jpg'))
        print(f"Created image {i} at {output_folder}/combined_{i}.jpg with {len(selected_images)} items.")

        with open(os.path.join(output_folder, f'combined_{i}.txt'), 'w') as f:
            for annotation in annotations:
                f.write(f"{annotation['class']} {annotation['bbox'][0]} {annotation['bbox'][1]} {annotation['bbox'][2]} {annotation['bbox'][3]}\n")

# Example usage:
create_dataset('archive')

Created image 0 at archive/combined_dataset/combined_0.jpg with 9 items.
Created image 1 at archive/combined_dataset/combined_1.jpg with 10 items.
Created image 2 at archive/combined_dataset/combined_2.jpg with 5 items.
Created image 3 at archive/combined_dataset/combined_3.jpg with 6 items.
Created image 4 at archive/combined_dataset/combined_4.jpg with 5 items.
Created image 5 at archive/combined_dataset/combined_5.jpg with 5 items.
Created image 6 at archive/combined_dataset/combined_6.jpg with 7 items.
Created image 7 at archive/combined_dataset/combined_7.jpg with 6 items.
Created image 8 at archive/combined_dataset/combined_8.jpg with 9 items.
Created image 9 at archive/combined_dataset/combined_9.jpg with 7 items.
Created image 10 at archive/combined_dataset/combined_10.jpg with 10 items.
Created image 11 at archive/combined_dataset/combined_11.jpg with 5 items.
Created image 12 at archive/combined_dataset/combined_12.jpg with 7 items.
Created image 13 at archive/combined_datase

In [ ]:
import os
import random
from PIL import Image, ImageEnhance

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        with Image.open(img_path) as img:
            img = img.convert("RGBA")
            images.append((img, filename, folder.split(os.sep)[-1]))
    return images

def check_overlap(new_box, boxes):
    for box in boxes:
        if (new_box[0] < box[2] and new_box[2] > box[0] and
            new_box[1] < box[3] and new_box[3] > box[1]):
            return True
    return False

def apply_transformations(img):
    angle = random.randint(-45, 45)
    img = img.rotate(angle, expand=True)
    enhancer = ImageEnhance.Brightness(img)
    factor = random.uniform(0.7, 1.3)
    img = enhancer.enhance(factor)
    return img

def combine_images(image_list, background_size, min_items):
    background = Image.new('RGBA', background_size, (255, 255, 255, 255))
    annotations = []
    occupied_areas = []

    for img, filename, label in image_list:
        img = apply_transformations(img)
        scale_x = background_size[0] / img.size[0]
        scale_y = background_size[1] / img.size[1]
        scale_factor = min(scale_x, scale_y, 1)
        new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
        img = img.resize(new_size)

        placed = False
        attempts = 0
        while not placed and attempts < 50:
            max_x = max(background_size[0] - img.size[0], 0)
            max_y = max(background_size[1] - img.size[1], 0)
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)
            new_box = [x, y, x + img.size[0], y + img.size[1]]
            if not check_overlap(new_box, occupied_areas):
                background.paste(img, (x, y), img)
                occupied_areas.append(new_box)
                annotations.append({
                    'class': label,
                    'bbox': new_box,
                    'filename': filename
                })
                placed = True
            attempts += 1

    return background.convert('RGB'), annotations

def create_dataset(base_folder, num_images=200, min_items_per_image=5, max_items_per_image=10, max_size=(2048, 2048)):
    categories = [os.path.join(base_folder, d) for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))]
    output_folder = os.path.join(base_folder, 'combined_dataset')
    os.makedirs(output_folder, exist_ok=True)

    for i in range(num_images):
        background_size = (random.randint(500, max_size[0]), random.randint(500, max_size[1]))
        selected_images = []
        while len(selected_images) < min_items_per_image:
            category_folder = random.choice(categories)
            images = load_images_from_folder(category_folder)
            if images:
                selected_image = random.choice(images)
                selected_images.append(selected_image)
        
        print(f"Debug: Attempt {i+1}, Number of selected images: {len(selected_images)}")  # Debug output

        if len(selected_images) < min_items_per_image:
            print("Skipping due to insufficient images after selection.")
            continue

        num_items = random.randint(min_items_per_image, min(len(selected_images), max_items_per_image))
        if len(selected_images) < num_items:
            print("Error in calculation of num_items, adjusting to length of selected_images.")
            num_items = len(selected_images)

        selected_images = random.sample(selected_images, num_items)

        combined_image, annotations = combine_images(selected_images, background_size, min_items_per_image)
        combined_image.save(os.path.join(output_folder, f'combined_{i}.jpg'))

        with open(os.path.join(output_folder, f'combined_{i}.txt'), 'w') as f:
            for annotation in annotations:
                f.write(f"{annotation['class']} {annotation['bbox'][0]} {annotation['bbox'][1]} {annotation['bbox'][2]} {annotation['bbox'][3]}\n")

# Example usage:
create_dataset('archive')